# with Groq Model

In [ ]:
#! touch "/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/agents.yaml"

In [ ]:
!pip install -r "/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/requirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.1/252.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.5/548.5 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.0 MB/s 

In [ ]:
from google.colab import userdata
grq_api = userdata.get('grog_api')
serper_api = userdata.get('serper_api')
mistral_api = userdata.get('mistral_api')
anthropic_api = userdata.get('anthropic_api')

In [ ]:
from groq import Groq
client = Groq(api_key=grq_api)

try:
    # Send prompt to Groq API
    completion = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": "are you free to use ? what is the usage limit for you?"}],
        temperature=0.7,
        max_completion_tokens=200,
        top_p=1,
        stream=True,
        stop=None,
    )

    # Stream output
    print("Response: ", end="")
    for chunk in completion:
        print(chunk.choices[0].delta.content or "", end="")

except Exception as e:
    print(f"\nError during API request: {e}")

Response: I'm a large language model, and my usage is governed by several factors. Here are some details:

1. **Fair use policy**: My usage is governed by a fair use policy, which means I can be used for personal, educational, or research purposes. However, I may not be used for commercial purposes without permission from my developers.
2. **Query limits**: I don't have a strict query limit, but I'm designed to handle a large volume of conversations. If I detect that you're making an excessive number of requests, I may slow down or limit my responses.
3. **Session limits**: I can maintain multiple conversations simultaneously, but each session has a limit on the number of turns (i.e., back-and-forth conversations). If we reach this limit, I may ask you to start a new conversation or take a break.
4. **Content limits**: I can process and respond to a wide range of content, including text, images, and other media. However

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# # Load environment variables
# from helper import load_env
# load_env()

import os
import yaml
from crewai import Agent, Task, Crew

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional

class MalwareSample(BaseModel):
    hash: str = Field(..., description="The hash of the malware sample (e.g., SHA-256, MD5).")
    source: str = Field(..., description="The source from where the malware intelligence was gathered.")
    classification: Optional[str] = Field(None, description="The classification of the malware (packer, dropper, loader).")
    description: Optional[str] = Field(None, description="Brief description of the malware behavior.")
    threat_level: Optional[str] = Field(None, description="Severity of the malware (Low, Medium, High).")

class OSINTReport(BaseModel):
    collected_samples: List[MalwareSample] = Field(..., description="A list of collected malware samples with metadata.")
    intelligence_sources: List[str] = Field(..., description="Sources used for gathering information.")

class ThreatAnalysis(BaseModel):
    indicators_of_compromise: List[str] = Field(..., description="List of IOCs such as IPs, domains, file hashes, and URLs.")
    behavior_analysis: str = Field(..., description="Detailed behavior analysis of the malware.")
    attack_techniques: List[str] = Field(..., description="MITRE ATT&CK techniques associated with the malware.")

class ClassificationReport(BaseModel):
    categorized_samples: List[MalwareSample] = Field(..., description="List of malware samples with classification details.")
    classification_summary: str = Field(..., description="Summary of classification criteria used.")

class ResearchSummary(BaseModel):
    key_findings: str = Field(..., description="High-level summary of all gathered intelligence and analysis.")
    recommended_next_steps: List[str] = Field(..., description="Suggested next steps for research and further analysis.")
    safe_analysis_guidelines: str = Field(..., description="Best practices for safely handling and analyzing the malware.")

class MalwareAnalysisOutput(BaseModel):
    osint_report: OSINTReport = Field(..., description="OSINT intelligence gathering report.")
    threat_analysis: ThreatAnalysis = Field(..., description="Technical analysis of the malware.")
    classification_report: ClassificationReport = Field(..., description="Categorization of the malware.")
    research_summary: ResearchSummary = Field(..., description="Final research findings and recommendations.")


In [ ]:
import yaml
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
os.environ['GROQ_API_KEY'] = grq_api
os.environ['SERPER_API_KEY'] = serper_api

# Define Groq LLM model
groq_llm = "groq/llama-3.1-8b-instant"

# Define file paths for YAML configurations
files = {
    'agents': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/agents.yaml',
    'tasks': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

# Creating Agents
osint_collector_agent = Agent(
    config=agents_config['osint_collector_agent'],
    tools=[SerperDevTool()],
    llm=groq_llm,
    max_iter=3,
    max_rpm=3,
    max_execution_time=30,
    verbose=True,
    max_retry_limit=1,
    respect_context_window=True,
    allow_delegation=False
)

threat_analyst_agent = Agent(
    config=agents_config['threat_analyst_agent'],
    tools=[SerperDevTool()],
    llm=groq_llm,
    max_iter=3,
    max_rpm=3,
    max_execution_time=30,
    verbose=True,
    max_retry_limit=1,
    respect_context_window=True,
    allow_delegation=False
)

malware_categorizer_agent = Agent(
    config=agents_config['malware_categorizer_agent'],
    tools=[ScrapeWebsiteTool()],
    llm=groq_llm,
    max_iter=3,
    max_rpm=3,
    max_execution_time=30,
    verbose=True,
    max_retry_limit=1,
    respect_context_window=True,
    allow_delegation=False
)

research_advisor_agent = Agent(
    config=agents_config['research_advisor_agent'],
    tools=[],
    llm=groq_llm,
    max_iter=3,
    max_rpm=3,
    max_execution_time=30,
    verbose=True,
    max_retry_limit=1,
    respect_context_window=True,
    allow_delegation=False
)

# Creating Tasks
search_malware_intelligence_task = Task(
    config=tasks_config['search_malware_intelligence'],
    agent=osint_collector_agent
)

analyze_malware_data_task = Task(
    config=tasks_config['analyze_malware_data'],
    agent=threat_analyst_agent
)

classify_malware_type_task = Task(
    config=tasks_config['classify_malware_type'],
    agent=malware_categorizer_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task]
)

generate_research_summary_task = Task(
    config=tasks_config['generate_research_summary'],
    agent=research_advisor_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task, classify_malware_type_task],
    output_pydantic=MalwareAnalysisOutput
)

# Creating Crew
malware_analysis_crew = Crew(
    agents=[
        osint_collector_agent,
        threat_analyst_agent,
        malware_categorizer_agent,
        research_advisor_agent
    ],
    tasks=[
        search_malware_intelligence_task,
        analyze_malware_data_task,
        classify_malware_type_task,
        generate_research_summary_task
    ],
    verbose=True
)

# Run the crew
if __name__ == "__main__":
    results = malware_analysis_crew.kickoff(inputs={'topic': 'find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums'})




# Agent: OSINT Researcher
## Task: Conduct an extensive OSINT search for information on malware families,  focusing on packers, droppers, and loaders. Gather data from sources like  VirusTotal, Hybrid Analysis, MITRE ATT&CK, and cybersecurity research forums.

 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for SerperDevToolSchema
search_query
  Input should be a valid string [type=string_type, input_value={'description': 'packers ...ers loaders VirusTotal'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A tool that can be used to search the internet with a search_query. Supports different search types: 'search' 

ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 4922, Requested 2967. Please try again in 18.881s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}





LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 4922, Requested 2967. Please try again in 18.881s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}



RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 4922, Requested 2967. Please try again in 18.881s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}


In [ ]:
import yaml
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
os.environ['GROQ_API_KEY'] = grq_api
os.environ['SERPER_API_KEY'] = serper_api

# Define Groq LLM model
groq_llm = "groq/llama-3.1-8b-instant"

# Define file paths for YAML configurations
files = {
    'agents': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/agents.yaml',
    'tasks': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

# Creating Agents
osint_collector_agent = Agent(
    config=agents_config['osint_collector_agent'],
    tools=[SerperDevTool()],
    llm=groq_llm,
    max_iter=3,
    max_rpm=3,
    max_execution_time=30,
    verbose=True,
    max_retry_limit=1,
    respect_context_window=True,
    allow_delegation=False
)

threat_analyst_agent = Agent(
    config=agents_config['threat_analyst_agent'],
    tools=[SerperDevTool()],
    llm=groq_llm,
    max_iter=3,
    max_rpm=3,
    max_execution_time=30,
    verbose=True,
    max_retry_limit=1,
    respect_context_window=True,
    allow_delegation=False
)

malware_categorizer_agent = Agent(
    config=agents_config['malware_categorizer_agent'],
    tools=[ScrapeWebsiteTool()],
    llm=groq_llm,
    max_iter=3,
    max_rpm=3,
    max_execution_time=30,
    verbose=True,
    max_retry_limit=1,
    respect_context_window=True,
    allow_delegation=False
)

research_advisor_agent = Agent(
    config=agents_config['research_advisor_agent'],
    tools=[],
    llm=groq_llm,
    max_iter=3,
    max_rpm=3,
    max_execution_time=30,
    verbose=True,
    max_retry_limit=1,
    respect_context_window=True,
    allow_delegation=False
)

# Creating Tasks
search_malware_intelligence_task = Task(
    config=tasks_config['search_malware_intelligence'],
    agent=osint_collector_agent
)

analyze_malware_data_task = Task(
    config=tasks_config['analyze_malware_data'],
    agent=threat_analyst_agent
)

classify_malware_type_task = Task(
    config=tasks_config['classify_malware_type'],
    agent=malware_categorizer_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task]
)

generate_research_summary_task = Task(
    config=tasks_config['generate_research_summary'],
    agent=research_advisor_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task, classify_malware_type_task],
    output_pydantic=MalwareAnalysisOutput
)

# Creating Crew
malware_analysis_crew = Crew(
    agents=[
        osint_collector_agent,
        threat_analyst_agent,
        malware_categorizer_agent,
        research_advisor_agent
    ],
    tasks=[
        search_malware_intelligence_task,
        analyze_malware_data_task,
        classify_malware_type_task,
        generate_research_summary_task
    ],
    verbose=True
)

# Run the crew
if __name__ == "__main__":
    results = malware_analysis_crew.kickoff(inputs={'topic': 'find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums'})




/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/selenium_scraping_tool/selenium_scraping_tool.py:26: PydanticDeprecatedSince20: Pydantic V1 style `@validator`

# Agent: find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums OSINT Researcher
## Task: Conduct an extensive OSINT search for information on malware families,  focusing on packers, droppers, and loaders. Gather data from sources like  VirusTotal, Hybrid Analysis, MITRE ATT&CK, and cybersecurity research forums.



# Agent: find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums OSINT Researcher
## Thought: Thought: to gather the necessary information on packers, droppers, and loaders, I should start by searching for known malware families and samples using trusted sources like VirusTotal and Hybrid Analysis.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"{\\\"keyword\\\": \\\"packers malware family\\\"}\"}"
## Tool Output: 
{'searchParameters': {'q': '{"keyword": "packers malware family"}', 'type

ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 5472, Requested 818. Please try again in 2.893999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}





LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 5472, Requested 818. Please try again in 2.893999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}



RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 5472, Requested 818. Please try again in 2.893999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}


In [ ]:
import time
from functools import wraps
import litellm  # Ensure you have this installed: pip install litellm
from crewai import LLM

def retry_with_exponential_backoff(func, max_retries=7, initial_delay=3):
    """Retries LLM requests with an increasing backoff delay."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        delay = initial_delay
        for attempt in range(max_retries):
            try:
                return func(*args, **kwargs)
            except litellm.RateLimitError as e:
                print(f"Rate limit exceeded. Retrying in {delay} seconds... (Attempt {attempt+1}/{max_retries})")
                if attempt == max_retries - 1:
                    raise  # Give up after max retries
                time.sleep(delay)
                delay *= 2  # Exponential backoff
    return wrapper


In [ ]:
class RetriableLLM(LLM):
    """LLM wrapper to include retry logic on completions."""

    @retry_with_exponential_backoff
    def complete(self, prompt: str):
        return super().complete(prompt)


In [ ]:


# Instantiate the LLM with retry logic
retriable_llm = RetriableLLM(model="groq/llama-3.1-8b-instant")



In [ ]:
import yaml
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
os.environ['GROQ_API_KEY'] = grq_api
os.environ['SERPER_API_KEY'] = serper_api

# Define file paths for YAML configurations
files = {
    'agents': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/agents.yaml',
    'tasks': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

# Creating Agents
osint_collector_agent = Agent(
    config=agents_config['osint_collector_agent'],
    tools=[SerperDevTool()],
    llm=retriable_llm
)

threat_analyst_agent = Agent(
    config=agents_config['threat_analyst_agent'],
    tools=[SerperDevTool()],
    llm=retriable_llm
)

malware_categorizer_agent = Agent(
    config=agents_config['malware_categorizer_agent'],
    tools=[ScrapeWebsiteTool()],
    llm=retriable_llm
)

research_advisor_agent = Agent(
    config=agents_config['research_advisor_agent'],
    tools=[],
    llm=retriable_llm
)

# Creating Tasks
search_malware_intelligence_task = Task(
    config=tasks_config['search_malware_intelligence'],
    agent=osint_collector_agent
)

analyze_malware_data_task = Task(
    config=tasks_config['analyze_malware_data'],
    agent=threat_analyst_agent
)

classify_malware_type_task = Task(
    config=tasks_config['classify_malware_type'],
    agent=malware_categorizer_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task]
)

generate_research_summary_task = Task(
    config=tasks_config['generate_research_summary'],
    agent=research_advisor_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task, classify_malware_type_task],
    output_pydantic=MalwareAnalysisOutput
)

# Creating Crew
malware_analysis_crew = Crew(
    agents=[
        osint_collector_agent,
        threat_analyst_agent,
        malware_categorizer_agent,
        research_advisor_agent
    ],
    tasks=[
        search_malware_intelligence_task,
        analyze_malware_data_task,
        classify_malware_type_task,
        generate_research_summary_task
    ],
    verbose=True
)

# Run the crew
if __name__ == "__main__":
    results = malware_analysis_crew.kickoff(inputs={'topic': 'find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums'})




# Agent: find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums OSINT Researcher
## Task: Conduct an extensive OSINT search for information on malware families,  focusing on packers, droppers, and loaders. Gather data from sources like  VirusTotal, Hybrid Analysis, MITRE ATT&CK, and cybersecurity research forums.



# Agent: find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums OSINT Researcher
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"packers droppers and loaders malware intelligence report\", \"search_type\": \"search\"}"
## Tool Output: 
{'searchParameters': {'q': 'packers droppers and loaders malware intelligence report', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Explained: Packer, Crypter, and Protector - ThreatDown', 'link': 'https://www.threatdown.com/bl

ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 4493, Requested 2840. Please try again in 13.328s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}





LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 4493, Requested 2840. Please try again in 13.328s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}



RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 4493, Requested 2840. Please try again in 13.328s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}


#with mistral model

In [ ]:
import yaml
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
os.environ['MISTRAL_API_KEY'] = mistral_api
os.environ['SERPER_API_KEY'] = serper_api


# Define file paths for YAML configurations
files = {
    'agents': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/agents.yaml',
    'tasks': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']
# retriable_llm = LLM(provider="mistral", model="mistral-large-latest", api_key=os.getenv("MISTRAL_API_KEY"))
retriable_llm = LLM(
    model="mistral/mistral-medium",
    temperature=0.7
)
#retriable_llm = "mistral/mistral-large-latest"

# Creating Agents
osint_collector_agent = Agent(
    config=agents_config['osint_collector_agent'],
    tools=[SerperDevTool()],
    llm=retriable_llm
)

threat_analyst_agent = Agent(
    config=agents_config['threat_analyst_agent'],
    tools=[SerperDevTool()],
    llm=retriable_llm
)

malware_categorizer_agent = Agent(
    config=agents_config['malware_categorizer_agent'],
    tools=[ScrapeWebsiteTool()],
    llm=retriable_llm
)

research_advisor_agent = Agent(
    config=agents_config['research_advisor_agent'],
    tools=[],
    llm=retriable_llm
)

# Creating Tasks
search_malware_intelligence_task = Task(
    config=tasks_config['search_malware_intelligence'],
    agent=osint_collector_agent
)

analyze_malware_data_task = Task(
    config=tasks_config['analyze_malware_data'],
    agent=threat_analyst_agent
)

classify_malware_type_task = Task(
    config=tasks_config['classify_malware_type'],
    agent=malware_categorizer_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task]
)

generate_research_summary_task = Task(
    config=tasks_config['generate_research_summary'],
    agent=research_advisor_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task, classify_malware_type_task],
    output_pydantic=MalwareAnalysisOutput
)

# Creating Crew
malware_analysis_crew = Crew(
    agents=[
        osint_collector_agent,
        threat_analyst_agent,
        malware_categorizer_agent,
        research_advisor_agent
    ],
    tasks=[
        search_malware_intelligence_task,
        analyze_malware_data_task,
        classify_malware_type_task,
        generate_research_summary_task
    ],
    verbose=True
)

# Run the crew
if __name__ == "__main__":
    results = malware_analysis_crew.kickoff(inputs={'topic': 'find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums'})




ERROR:opentelemetry.sdk.trace.export:Exception while exporting Span batch.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 464, in _make_request
    self._validate_conn(conn)
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 1093, in _validate_conn
    conn.connect()
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 741, in connect
    sock_and_verified = _ssl_wrap_socket_and_match_hostname(
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 920, in _ssl_wrap_socket_and_match_hostname
    ssl_sock = ssl_wrap_socket(
               ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/ssl_.py", line 460, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(sock, context, tls_in_tls, server_hostname)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

# Agent: find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums OSINT Researcher
## Task: Conduct an extensive OSINT search for information on malware families,  focusing on packers, droppers, and loaders. Gather data from sources like  VirusTotal, Hybrid Analysis, MITRE ATT&CK, and cybersecurity research forums.

 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for SerperDevToolSchema
search_query
  Field required [type=missing, input_value={'search\\_query': 'packe...te:hybrid-analysis.com'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A 

ERROR:root:LiteLLM call failed: litellm.BadRequestError: MistralException - Error code: 400 - {'object': 'error', 'message': 'Expected last role User or Tool (or Assistant with prefix True) for serving but got assistant', 'type': 'invalid_request_error', 'param': None, 'code': None}




LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.BadRequestError: MistralException - Error code: 400 - {'object': 'error', 'message': 'Expected last role User or Tool (or Assistant with prefix True) for serving but got assistant', 'type': 'invalid_request_error', 'param': None, 'code': None}


BadRequestError: litellm.BadRequestError: MistralException - Error code: 400 - {'object': 'error', 'message': 'Expected last role User or Tool (or Assistant with prefix True) for serving but got assistant', 'type': 'invalid_request_error', 'param': None, 'code': None}

#Anthropic model (Claude)

In [ ]:
import yaml
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
os.environ['ANTHROPIC_API_KEY'] = anthropic_api
os.environ['SERPER_API_KEY'] = serper_api


# Define file paths for YAML configurations
files = {
    'agents': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/agents.yaml',
    'tasks': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

retriable_llm = LLM(
    model="anthropic/claude-3-sonnet-20240229-v1:0",
    temperature=0.7
)


# Creating Agents
osint_collector_agent = Agent(
    config=agents_config['osint_collector_agent'],
    tools=[SerperDevTool()],
    llm=retriable_llm
)

threat_analyst_agent = Agent(
    config=agents_config['threat_analyst_agent'],
    tools=[SerperDevTool()],
    llm=retriable_llm
)

malware_categorizer_agent = Agent(
    config=agents_config['malware_categorizer_agent'],
    tools=[ScrapeWebsiteTool()],
    llm=retriable_llm
)

research_advisor_agent = Agent(
    config=agents_config['research_advisor_agent'],
    tools=[],
    llm=retriable_llm
)

# Creating Tasks
search_malware_intelligence_task = Task(
    config=tasks_config['search_malware_intelligence'],
    agent=osint_collector_agent
)

analyze_malware_data_task = Task(
    config=tasks_config['analyze_malware_data'],
    agent=threat_analyst_agent
)

classify_malware_type_task = Task(
    config=tasks_config['classify_malware_type'],
    agent=malware_categorizer_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task]
)

generate_research_summary_task = Task(
    config=tasks_config['generate_research_summary'],
    agent=research_advisor_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task, classify_malware_type_task],
    output_pydantic=MalwareAnalysisOutput
)

# Creating Crew
malware_analysis_crew = Crew(
    agents=[
        osint_collector_agent,
        threat_analyst_agent,
        malware_categorizer_agent,
        research_advisor_agent
    ],
    tasks=[
        search_malware_intelligence_task,
        analyze_malware_data_task,
        classify_malware_type_task,
        generate_research_summary_task
    ],
    verbose=True
)

# Run the crew
if __name__ == "__main__":
    results = malware_analysis_crew.kickoff(inputs={'topic': 'find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums'})




ERROR:root:LiteLLM call failed: litellm.BadRequestError: AnthropicException - {"type":"error","error":{"type":"invalid_request_error","message":"Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits."}}


# Agent: find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums OSINT Researcher
## Task: Conduct an extensive OSINT search for information on malware families,  focusing on packers, droppers, and loaders. Gather data from sources like  VirusTotal, Hybrid Analysis, MITRE ATT&CK, and cybersecurity research forums.



LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.BadRequestError: AnthropicException - {"type":"error","error":{"type":"invalid_request_error","message":"Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits."}}


BadRequestError: litellm.BadRequestError: AnthropicException - {"type":"error","error":{"type":"invalid_request_error","message":"Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits."}}

#Agents using AutoGen

In [ ]:
# pip install autogen-agentchat["mistral"]~=0.2 # for Mistral AI client
# pip install autogen-agentchat["anthropic"]~=0.2 # for Anthropic client
# pip install autogen-agentchat["together"]~=0.2 # for Together.AI client
# pip install autogen-agentchat["groq"]~=0.2 # for Groq client

In [ ]:
[
    {
        "model": "your anthropic model name",
        "api_key": "your Anthropic api_key",
        "api_type": "anthropic"
    },
    {
        "model": "your mistral model name",
        "api_key": "your Mistral AI api_key",
        "api_type": "mistral"
    },
    {
        "model": "your together.ai model name",
        "api_key": "your Together.AI api_key",
        "api_type": "together"
    },
    {
        "model": "your groq model name",
        "api_key": os.environ['OPENAI_API_KEY'],
        "api_type": "groq"
    }
]

In [ ]:
# [
#   {
#     "model": "gpt-4",
#     "api_key": os.environ['OPENAI_API_KEY']
#     "api_rate_limit": 60.0, // Set to allow up to 60 API requests per second.
#   }
# ]

In [ ]:
!pip install autogen


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import userdata
import os
grq_api = userdata.get('grog_api')
os.environ['GROQ_API_KEY'] = grq_api
# groq_llm = "groq/llama-3.1-8b-instant"

In [ ]:
llm_config = {
        "model": "llama-3.1-8b-instant",
        "api_key": os.environ['GROQ_API_KEY'],
        "api_type": "groq"
    }

In [ ]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)


In [ ]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

{'content': 'A man walked into a library and asked the librarian, "Do you have any books on Pavlov\'s dogs and Schrödinger\'s cat?" The librarian replied, "It rings a bell, but I\'m not sure if it\'s here or not."', 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': None}


In [ ]:
sec_exp_1 = ConversableAgent(
    name="malware_analyst",
    system_message=
    "Your name is malware_analyst and you are an expert malware analyst.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

sec_exp_2 = ConversableAgent(
    name="malware_researcher",
    system_message=
    "Your name is malware_researcher and you research on malware on a high level including all tools you know "
    "Support malware analyst findings, questions or any other information he is providing.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [ ]:
chat_result = sec_exp_1.initiate_chat(
    recipient=sec_exp_2,
    message="I'm malware analyst. malware_researcher, let's discuss about the latest types of malware recently discovered.",
    max_turns=2,
)

malware_analyst (to malware_researcher):

I'm malware analyst. malware_researcher, let's discuss about the latest types of malware recently discovered.

--------------------------------------------------------------------------------
malware_researcher (to malware_analyst):

Nice to collaborate with you, fellow malware analyst. I've been tracking various malware families and variants lately, and I'd like to share some insights with you. 

**1.** Recently, I came across a strain of malware known as "Gootkit" (also known as "Gootloader" or "Rig EK"). It's a banking trojan and ransomware downloader that injects malicious code into legitimate websites. This malware is quite stealthy, using DNS tunneling to communicate with the command and control server.

**2.** Another malware family that caught my attention is "Clop" (also known as "Play." or "Dridex"). This is a ransomware strain that employs a "pay-to-play" model, targeting various industries, including healthcare, finance, and manufac

In [ ]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm malware analyst. malware_researcher, let's discuss about the "
             'latest types of malware recently discovered.',
  'name': 'malware_analyst',
  'role': 'assistant'},
 {'content': "Nice to collaborate with you, fellow malware analyst. I've been "
             "tracking various malware families and variants lately, and I'd "
             'like to share some insights with you. \n'
             '\n'
             '**1.** Recently, I came across a strain of malware known as '
             '"Gootkit" (also known as "Gootloader" or "Rig EK"). It\'s a '
             'banking trojan and ransomware downloader that injects malicious '
             'code into legitimate websites. This malware is quite stealthy, '
             'using DNS tunneling to communicate with the command and control '
             'server.\n'
             '\n'
             '**2.** Another malware family that caught my attention is "Clop" '
             '(also known as "Play." or "Dridex"). This i

In [ ]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'llama-3.1-8b-instant': {'completion_tokens': 1611,
                                                               'cost': 0.0,
                                                               'prompt_tokens': 1662,
                                                               'total_tokens': 3273},
                                      'total_cost': 0.0},
 'usage_including_cached_inference': {'llama-3.1-8b-instant': {'completion_tokens': 1611,
                                                               'cost': 0.0,
                                                               'prompt_tokens': 1662,
                                                               'total_tokens': 3273},
                                      'total_cost': 0.0}}


In [ ]:
pprint.pprint(chat_result.summary)

("It's fantastic to collaborate with a fellow malware analyst and share our "
 'knowledge and insights. Your additional information on each of the malware '
 'families we discussed earlier is very informative and relevant.\n'
 '\n'
 '**1. Gootkit**\n'
 '\n'
 'Your mention of the "web injection" technique used by Gootkit is a great '
 'point. This technique can make it even more challenging for security '
 'solutions to detect and mitigate the malicious activity. I agree that the '
 '"Rig EK" variant is particularly concerning, as it can bypass certain '
 'security measures and inject malware into websites.\n'
 '\n'
 '**2. Clop**\n'
 '\n'
 "Yes, Clop's use of exploit kits (EKs) to gain initial access to victims' "
 'systems and then spread through the network is a notable aspect of its '
 'behavior. Its ability to evade detection using techniques such as ring zero '
 'protection is also a significant concern.\n'
 '\n'
 '**3. AgentTesla**\n'
 '\n'
 "I completely agree with your assessmen

# Poject ideas for initial workflow and structure

## Idea -1 (we can implemet this using various agentic frame work like autogen and crew ai)

🔹 Step-by-Step Workflow: Malware Analysis & Research AI Agent System
🟢 Step 1: Input Selection (User Provides One or More of the Following)
Malware Sample (Binary file, EXE, DLL, etc.)
Malware Hash (MD5, SHA-256)
CVE ID (e.g., CVE-2023-12345)
CWE ID (e.g., CWE-94 for Code Injection)
Threat Actor Name / Campaign (e.g., Lazarus Group, Emotet)
Decision Making:

If a malware sample or hash is provided → Focus on malware analysis first and later find related CVEs/CWEs.
If a CVE or CWE is provided → Focus on OSINT gathering first and later find associated malware exploiting this weakness.
If a threat actor is mentioned → Start by gathering OSINT intelligence on their campaigns and tactics.
🔵 Step 2: Automated Malware Analysis (If Sample or Hash is Given)
A. Static Analysis (No Execution)
🚀 Agent: Static Analysis Agent
🔹 Task: Extract information without executing the malware.
✅ Outputs:

File metadata (size, sections, entropy)
Embedded strings (e.g., URLs, commands, API calls)
Disassembly analysis (detect obfuscation, packers)
Signature matching (YARA rules, AV detections)
B. Dynamic Analysis (Behavior Execution in Sandbox)
🚀 Agent: Dynamic Analysis Agent
🔹 Task: Execute malware safely in a controlled VM environment.
✅ Outputs:

Process creation and network activity
Registry modifications and file system changes
Detection of persistence mechanisms (e.g., scheduled tasks, startup entries)
API calls (e.g., CreateRemoteThread, VirtualAllocEx)
🟠 Step 3: OSINT Intelligence Gathering (CVE, CWE, MITRE ATT&CK, Threat Intelligence)
A. Linking Malware to Known CVEs & CWEs
🚀 Agent: OSINT Correlation Agent
🔹 Task: Find known vulnerabilities that malware exploits.
✅ Outputs:

Related CVE IDs (e.g., CVE-2023-4567 linked to a malware sample)
Weaknesses from CWE Database
Mapping to MITRE ATT&CK Techniques (e.g., T1059: Command and Scripting Interpreter)
Example:
If a malware sample exploits CVE-2023-12345, the agent fetches:

Patch status
Exploits available on GitHub
Threat actor campaigns using it
B. Gathering IOCs from OSINT Sources
🚀 Agent: Threat Intelligence OSINT Agent
🔹 Task: Fetch IOCs (Indicators of Compromise) from various sources.
✅ Outputs:

VirusTotal, Hybrid Analysis reports
Shodan searches for compromised hosts
Dark web mentions (if available)
🟡 Step 4: AI-Assisted Cybersecurity Research & Reporting
A. Summarizing Analysis & Generating Reports
🚀 Agent: Research Summarization Agent
🔹 Task: Convert technical data into an analyst-friendly report.
✅ Outputs:

Executive Summary
Technical Analysis
IOC List & MITRE ATT&CK Mapping
Recommended Mitigation Steps
B. Reviewing & Refining Findings
🚀 Agent: Critic Agent
🔹 Task: Review report accuracy and suggest improvements.
✅ Outputs:

Detect missing information
Provide alternate recommendations
Ensure report is concise and readable
🎯 Final Output
After all these steps, the system provides:

Malware Analysis Report
OSINT-Based Threat Intelligence Report
CVE & CWE Analysis
MITRE ATT&CK Mapping
List of IOCs for Security Teams
🔍 Why This Workflow Solves Real-World Challenges
🔹 Pain Point #1: Malware Analysis is Time-Consuming
✅ Solution: Automate both static and dynamic analysis with AI.

🔹 Pain Point #2: Finding CVE/CWE Links is Manual
✅ Solution: Auto-map malware to known vulnerabilities and weaknesses.

🔹 Pain Point #3: OSINT Research is Overwhelming
✅ Solution: Auto-fetch threat intelligence from public sources.

🔹 Pain Point #4: Reports Take Too Long to Create
✅ Solution: AI summarizes findings & generates detailed reports.



In [ ]:
from autogen import ConversableAgent

# Creating the Input Agent
input_agent = ConversableAgent(
    name="malware_input_agent",
    system_message="You are a cybersecurity assistant. "
                   "Ask the user for either a malware hash, CVE ID, or CWE ID. "
                   "Once you receive the input, decide whether to proceed with malware analysis or OSINT intelligence gathering.",
    human_input_mode="ALWAYS",  # Always waiting for user input
    default_auto_reply="Please provide a malware hash, CVE ID, or CWE number."
)

# Function to process user input
def process_input(user_input):
    if user_input.startswith("CVE-"):
        return f"Received CVE: {user_input}. Forwarding to OSINT Intelligence Agent."
    elif user_input.startswith("CWE-"):
        return f"Received CWE: {user_input}. Forwarding to OSINT Intelligence Agent."
    elif len(user_input) in [32, 40, 64]:  # Typical hash lengths (MD5, SHA1, SHA256)
        return f"Received malware hash: {user_input}. Forwarding to Malware Analysis Agent."
    else:
        return "Invalid input. Please enter a valid hash, CVE, or CWE."

# Start conversation
while True:
    user_input = input("Enter a malware hash, CVE ID, or CWE ID: ")
    response = process_input(user_input)
    print(response)

    if "Forwarding" in response:
        break  # Stop when a valid input is received


In [ ]:
from autogen import AssistantAgent

# OSINT Intelligence Agent
osint_agent = AssistantAgent(
    name="osint_intelligence_agent",
    system_message="You are an OSINT intelligence agent. Your task is to gather details on a provided CVE or CWE number from known databases.",
    human_input_mode="NEVER",  # Fully automated
)

# Function to fetch dummy data (Replace with real OSINT sources later)
def fetch_osint_data(cve_or_cwe):
    if cve_or_cwe.startswith("CVE-"):
        return f"Details on {cve_or_cwe}: It is a critical vulnerability affecting X software..."
    elif cve_or_cwe.startswith("CWE-"):
        return f"Details on {cve_or_cwe}: This weakness is commonly exploited in Y scenarios..."
    else:
        return "Invalid input. Please provide a valid CVE or CWE number."

# Example usage
user_input = "CVE-2023-12345"  # Example input
osint_data = fetch_osint_data(user_input)
print(osint_data)


In [ ]:
# Summarizer Agent
summarizer_agent = AssistantAgent(
    name="summarizer_agent",
    system_message="You are a summarizer agent. Your task is to generate a structured summary from OSINT intelligence and malware analysis reports.",
    human_input_mode="NEVER",  # Fully automated
)

# Function to generate a dummy summary
def generate_summary(osint_data, malware_data=""):
    summary = f"🔍 **Summary Report** 🔍\n\n"
    summary += f"**OSINT Findings:**\n{osint_data}\n\n"
    summary += f"**Malware Analysis Findings:**\n{malware_data if malware_data else 'No malware analysis data available.'}\n\n"
    summary += "✅ This report combines OSINT intelligence and malware insights for security research."
    return summary

# Example usage
malware_analysis_report = "Detected as a Trojan variant. Exhibits C2 communication."
summary = generate_summary(osint_data, malware_analysis_report)
print(summary)


In [ ]:
from autogen import AssistantAgent

# Malware Analysis Agent
malware_analysis_agent = AssistantAgent(
    name="malware_analysis_agent",
    system_message="You are a cybersecurity malware analysis agent. "
                   "Your task is to analyze provided malware hashes or files, "
                   "fetching intelligence or performing static analysis.",
    human_input_mode="NEVER",  # Fully automated
)

# Function to simulate basic malware analysis
def analyze_malware(input_data, file_mode=False):
    if file_mode:
        return f"Analyzing file: {input_data}. Extracting metadata, imports, and strings..."
    else:
        return f"Looking up hash {input_data}. Checking against known databases for malware signatures..."

# Example usage
hash_input = "5d41402abc4b2a76b9719d911017c592"  # Example MD5 hash
malware_report = analyze_malware(hash_input)
print(malware_report)
